In [1]:
import tensorflow as tf
from tensorflow.image import non_max_suppression
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import imutils
import pickle
import cv2
import os
from datetime import datetime

In [2]:
start_time = datetime.now()
model = load_model("model/licence-plate_v2.h5")
lb = pickle.loads(open("label_encoder/label_encoder_cars.pickle", "rb").read())

In [28]:
image = cv2.imread("car/images/Cars0.png")
image = imutils.resize(image, width=500)
print(image.shape)
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
ss.setBaseImage(image)
ss.switchToSelectiveSearchFast()
rect = ss.process()

(268, 500, 3)


In [29]:
proposals = []
boxes = []
for x, y, w, h in rect[:200]:
    roi = image[y : y + h, x : x + w]
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    roi = cv2.resize(roi, (224, 224), interpolation=cv2.INTER_CUBIC)
    roi = img_to_array(roi)
    # roi = preprocess_input(roi)
    proposals.append(roi)
    boxes.append((x, y, x + w, y + h))

In [30]:
proposals = np.array(proposals)
boxes = np.array(boxes)
print(proposals.shape)
proba = model.predict(proposals)

(200, 224, 224, 3)


In [31]:
proba

array([[1.04678703e-10, 1.00000000e+00],
       [1.32494451e-05, 9.99986768e-01],
       [7.46042428e-09, 1.00000000e+00],
       [5.70059381e-03, 9.94299412e-01],
       [1.84669498e-05, 9.99981523e-01],
       [1.26278111e-11, 1.00000000e+00],
       [8.49052303e-05, 9.99915123e-01],
       [1.83846638e-09, 1.00000000e+00],
       [4.57721014e-11, 1.00000000e+00],
       [1.44820529e-04, 9.99855161e-01],
       [1.38158257e-05, 9.99986172e-01],
       [9.02689035e-09, 1.00000000e+00],
       [7.30645877e-10, 1.00000000e+00],
       [7.69256440e-06, 9.99992251e-01],
       [3.40201450e-10, 1.00000000e+00],
       [5.96864368e-07, 9.99999404e-01],
       [1.52087019e-08, 1.00000000e+00],
       [9.75622605e-09, 1.00000000e+00],
       [3.37391781e-10, 1.00000000e+00],
       [1.73425888e-05, 9.99982715e-01],
       [1.54296367e-10, 1.00000000e+00],
       [1.10264064e-11, 1.00000000e+00],
       [6.40558184e-08, 9.99999881e-01],
       [7.24530473e-05, 9.99927521e-01],
       [4.011159

In [32]:
print("[INFO] applying NMS...")
labels = lb.classes_[np.argmax(proba, axis=1)]
print(labels)
idxs = np.where(labels == "licence")[0]
print(idxs)
# use the indexes to extract all bounding boxes and associated class
# label probabilities associated with the "raccoon" class
boxes = boxes[idxs]
proba = proba[idxs][:, 0]
# further filter indexes by enforcing a minimum prediction
# probability be met
print(proba)
idxs = np.where(proba >= 0.8)
print(idxs)
boxes = boxes[idxs]
proba = proba[idxs]


[INFO] applying NMS...
['no_licence' 'no_licence' 'no_licence' 'no_licence' 'no_licence'
 'no_licence' 'no_licence' 'no_licence' 'no_licence' 'no_licence'
 'no_licence' 'no_licence' 'no_licence' 'no_licence' 'no_licence'
 'no_licence' 'no_licence' 'no_licence' 'no_licence' 'no_licence'
 'no_licence' 'no_licence' 'no_licence' 'no_licence' 'no_licence'
 'no_licence' 'licence' 'no_licence' 'no_licence' 'no_licence'
 'no_licence' 'no_licence' 'no_licence' 'no_licence' 'no_licence'
 'no_licence' 'no_licence' 'no_licence' 'no_licence' 'no_licence'
 'no_licence' 'no_licence' 'no_licence' 'no_licence' 'no_licence'
 'no_licence' 'no_licence' 'no_licence' 'no_licence' 'no_licence'
 'no_licence' 'no_licence' 'no_licence' 'no_licence' 'no_licence'
 'no_licence' 'no_licence' 'no_licence' 'no_licence' 'no_licence'
 'no_licence' 'no_licence' 'no_licence' 'no_licence' 'no_licence'
 'no_licence' 'no_licence' 'no_licence' 'no_licence' 'no_licence'
 'no_licence' 'no_licence' 'licence' 'no_licence' 'licen

In [33]:
# clone the original image so that we can draw on it
clone = image.copy()
# loop over the bounding boxes and associated probabilities
for (box, prob) in zip(boxes, proba):
    # draw the bounding box, label, and probability on the image
    (startX, startY, endX, endY) = box
    # cv2.rectangle(clone, (startX, startY), (endX, endY), (0, 255, 0), 2)
    # y = startY - 10 if startY - 10 > 10 else startY + 10
    # text = "Raccoon: {:.2f}%".format(prob * 100)
    # cv2.putText(
    #     clone, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2
    # )
# show the output after *before* running NMS
# cv2.imshow("Before NMS", clone)
# cv2.waitKey(100)
# cv2.destroyAllWindows()

In [34]:
from datetime import datetime
import os
batch_id = str(datetime.time(datetime.now()))
OutputPath = os.path.join("OCR", f"{batch_id}.jpg")
boxIdxs = non_max_suppression(boxes, proba, max_output_size=3)
boxes = tf.gather(boxes, boxIdxs)
# loop over the bounding box indexes
print(boxes)
for i in boxIdxs:
    print(i)
    # draw the bounding box, label, and probability on the image
(startX, startY, endX, endY) = boxes[0]
cv2.rectangle(image, (startX, startY), (endX, endY), (0, 255, 0), 2)
y = startY - 10 if startY - 10 > 10 else startY + 10
text = "Raccoon: {:.2f}%".format(proba[i] * 100)
cv2.putText(
    image, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2
)
cropped = image[startY:endY,startX:endX]
cv2.imwrite(OutputPath, cropped)
# show the output image *after* running NMS

tf.Tensor(
[[249 124 417 176]
 [180 118 440 192]
 [144 113 336 176]], shape=(3, 4), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)


True

In [37]:
import time
start_time = datetime.now()
time.sleep(5)
end_time = datetime.now()
time_taken = end_time - start_time
print(f"Time taken for infer: {time_taken.seconds} seconds")

Time taken for infer: 5 seconds
